In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 13596



### keep drop rename


In [2]:
sas.sasdata("stocks","sashelp").head()

,Stock,Date,Open,High,Low,Close,Volume,AdjClose
0,IBM,2005-12-01,89.15,89.92,81.56,82.20,5976252.0,81.37
1,IBM,2005-11-01,81.85,89.94,80.64,88.90,5556471.0,88.01
2,IBM,2005-10-03,80.22,84.60,78.70,81.88,7019666.0,80.86
3,IBM,2005-09-01,80.16,82.11,76.93,80.22,5772280.0,79.22
4,IBM,2005-08-01,83.00,84.20,79.87,80.62,4801386.0,79.62


In [3]:
# data set options change data that are read or write
# can be used in data statements, and as part of proc data statement
sas.submitLST(f"""
data _df (keep = stock date close high);
    set sashelp.stocks (keep = stock date close high volume);
              
proc print data = _df (drop = high obs = 2);
run;
proc print data = _df (rename = (close = price) drop = high obs = 2);
run;

""")

Obs,Stock,Date,Close
1,IBM,01DEC05,$82.20
2,IBM,01NOV05,$88.90
Obs,Stock,Date,price
1,IBM,01DEC05,$82.20
2,IBM,01NOV05,$88.90


### obs


In [4]:
sas.submitLST(f"""
proc print data=sashelp.class(firstobs=2 obs=3);
run;
""")

Obs,Name,Sex,Age,Height,Weight
2,Alice,F,13,56.5,84
3,Barbara,F,13,65.3,98


### in


In [5]:
sas.submitLST(f"""
data _a;
    set sashelp.class;
    where name not in ('Alfred', 'Alice', 'Barbara');
run;
proc sort data=_a;
    by name;
run;

data _b;
    set sashelp.class;
    where name not in ('Carol', 'Henry');
run;
proc sort data=_b;
    by name;
run;
              
""", method="listonly")


In [6]:
# in option creates a variable that indicates if a record is in the data set
sas.submitLST(f"""
data _c;
    merge _a (in=a) _b (in=b);
    by name;
    a2 = a;
    b2 = b;
    if a or b;
run;
              
proc print data = _c (obs = 5);
run;
              
proc freq data = _c noprint;
    tables a2 * b2 / out = _d;
run;
              
proc print data = _d;
run;
""")

### where

In [7]:
sas.submitLST(f"""
proc print data=sashelp.class(where=(sex='M') obs=2);
run;
""")

Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
5,Henry,M,14,63.5,102.5


In [8]:
sas.submitLST(f"""
data a (where=(age>15)) b (where=(sex='F'));
    set sashelp.class;
run;
              
proc sql;
    select count(*) as a from a;
run;

proc sql;
    select count(*) as b from b;
run;
""")

a
1
b
9
